In [81]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
from selenium import webdriver
import os
import time
import re

In [82]:
# channel videos page url
url = 'https://www.youtube.com/channel/UCMmaBzfCCwZ2KqaBJjkj0fw/videos'

In [83]:
# load chrome driver and open url
driver = webdriver.Chrome(os.getcwd() + '/chromedriver')
driver.get(url)

# get channel name
channel_name = driver.find_element_by_xpath('//*[@class="style-scope ytd-channel-name"]').text
print('Channel name:', channel_name)

# list to store all links in the channel playlist
links = []

# get to items tag and start scrolling
video_section = driver.find_element_by_xpath('//*[@id="items"]')
driver.execute_script("arguments[0].scrollIntoView();", video_section)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
time.sleep(5)
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait to load page
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height
    
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

# get video links from thumbnail and append in list
videos = driver.find_elements_by_xpath('//*[@id="thumbnail"]')
for i in range(len(videos)):
    link = videos[i].get_attribute("href")
    links.append(link)

driver.close()

Channel name: Kings and Generals


In [1]:
video_comments = {}

print("Total videos in Channel -->", url, ":", len(links))

# iterate over links and save comments
for i, link in enumerate(links):
    print("video", i+1, "link:", link)
    # load driver and open video page
    driver = webdriver.Chrome(os.getcwd() + '/chromedriver')
    driver.get(link)
    time.sleep(5)

    # traverse to comment section
    comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    # get scroll height
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load page
        time.sleep(5)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # use regex to traverse emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)

    # get names of people who commented
    names = driver.find_elements_by_xpath('//*[@id="author-text"]')

    # get comment texts
    comments = driver.find_elements_by_xpath('//*[@id="content-text"]')

    # iterate over comments and save in video_comments dictionary
    for i in range(len(names)):
        username = names[i].text
        comment = comments[i].text 
        video_comments.update({username : comment})

    driver.close()

In [85]:
# convert dictionary to dataframe and save in a csv file
channel_name = channel_name.lower().replace(' ', '_')
file = channel_name + '_comments.csv'
df = pd.DataFrame(list(video_comments.items()),columns = ['Username','Comment'])
df.to_csv(file, index=False)